In [1]:
import os
from dotenv import load_dotenv
import sys

script_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

from src.paths import PARENT_DIR

In [2]:
import hopsworks

In [3]:
print(hopsworks.__version__)

3.4.2


In [4]:
# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
HOPSWORKS_PROJECT_NAME = 'taxi_demand_calvin'

In [5]:

from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = 2023 #datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local 

In [6]:
print(f'{len(rides)=:,}')

len(rides)=77,964,130


In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:06<00:00, 43.80it/s]


In [47]:
import hopsworks

In [48]:
print(hopsworks.__version__)

3.4.2


In [49]:
# IcjMMvdfzZBOlCbE.mGsaeXANM7Y0zDxEwNUIRYkBqoEFLntZnC2oxTCRfbAfM1KGrf9C919b3XrFYECI

In [66]:
# HOPSWORKS_API_KEY='IcjMMvdfzZBOlCbE.mGsaeXANM7Y0zDxEwNUIRYkBqoEFLntZnC2oxTCRfbAfM1KGrf9C919b3XrFYECI'

In [54]:
HOPSWORKS_PROJECT_NAME

'taxi_demand_calvin'

In [55]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/578708


In [56]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [57]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [64]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [65]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/578708/fs/574531/fg/669031


Uploading Dataframe: 0.00% |          | Rows 0/4642800 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/578708/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f91b01bfb50>, None)

In [72]:
import xgboost